In [5]:
import toml
# from abetam.data.canada import Provinces

config = toml.load("../copper/scenarios/BAU_scenario/config.toml")

config["Simulation_Settings"]["ap"] = ["Ontario"]
config["Simulation_Settings"]["ap"], config["Simulation_Settings"]["aba"]

(['Ontario'],
 ['Alberta.a', 'Saskatchewan.a', 'Manitoba.a', 'Ontario.a', 'Ontario.b'])

In [6]:
config["Simulation_Settings"]["aba"] = [ba for ba in config["Simulation_Settings"]["aba"] if any(prov in ba for prov in config["Simulation_Settings"]["ap"])]
config["Simulation_Settings"]["aba"]

['Ontario.a', 'Ontario.b']

In [8]:
import git

In [16]:
from pathlib import Path
# dir(git.Repo())
root_dir = git.Repo().working_dir
abetam_dir = Path(root_dir) / "abetam"


WindowsPath('c:/src/canadopt/abetam')

In [18]:
git.Repo(".").submodules

[]